In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

def plot_histograms_and_scalars(logs_dir='raw_np', plots_dir='plots', bins=50):
    """
    Generate bar histograms for activations and line plots for training statistics.
    """
    # Ensure plots directory exists
    os.makedirs(plots_dir, exist_ok=True)
    
    # Iterate through each model directory in logs
    for model_name in os.listdir(logs_dir):
        model_log_dir = os.path.join(logs_dir, model_name)
        if not os.path.isdir(model_log_dir):
            continue
            
        # Create model-specific plot directories
        model_plot_dir = os.path.join(plots_dir, model_name)
        activations_plot_dir = os.path.join(model_plot_dir, 'activations')
        scalars_plot_dir = os.path.join(model_plot_dir, 'training')
        act_params_plot_dir = os.path.join(model_plot_dir, 'activations_parameters')
        os.makedirs(activations_plot_dir, exist_ok=True)
        os.makedirs(scalars_plot_dir, exist_ok=True)
        os.makedirs(act_params_plot_dir, exist_ok=True)
        
        # Plot histograms for activations
        acts_dir = os.path.join(model_log_dir, 'activations')
        for file in os.listdir(acts_dir):
            if file.endswith('.npz'):
                file_path = os.path.join(acts_dir, file)
                data = np.load(file_path)
                act = data["activations"]
                params = data["params"]
                layer_name = file.split('_')[0]
                epoch = file.split('_')[2].split('.')[0]
                
                plt.figure(figsize=(8, 6))
                plt.hist(act.flatten(), bins=bins, density=True)
                subtitle = f"\nLearnable parameters values = {", ".join(f"{x:.6f}" for x in params)}" if params.size >= 1 else ""
                plt.title(f'Activation Histogram for {layer_name} at Epoch {epoch}{subtitle}\n{model_name}')
                plt.xlabel('Activation Value')
                plt.ylabel('Density')
                plt.grid(True, alpha=0.3)
                plt.savefig(os.path.join(activations_plot_dir, f'{layer_name}_epoch_{epoch}.png'))
                plt.close()
                if "quant" in model_name:
                    a = np.unique(act.flatten())
                    print(f"[{model_name}] Unique activations of {layer_name} at Epoch {epoch} - {len(a)}:", a)

        # Plot training statistics
        scalars_file = os.path.join(model_log_dir, 'training_stats.npz')
        if os.path.exists(scalars_file):
            data = np.load(scalars_file)
            epochs = data['epoch']
            
            # Plot losses
            plt.figure(figsize=(8, 6))
            plt.plot(epochs, data['train_loss'], label='Train Loss')
            plt.plot(epochs, data['eval_loss'], label='Eval Loss')
            plt.title(f'Losses vs. Epoch\n{model_name}')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.savefig(os.path.join(scalars_plot_dir, 'losses.png'))
            plt.close()
            
            # Plot accuracies
            plt.figure(figsize=(8, 6))
            plt.plot(epochs, data['train_acc'], label='Train Accuracy')
            plt.plot(epochs, data['eval_acc'], label='Eval Accuracy')
            plt.title(f'Accuracies vs. Epoch\n{model_name}')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.savefig(os.path.join(scalars_plot_dir, 'accuracies.png'))
            plt.close()
            
            # Plot learning rate
            plt.figure(figsize=(8, 6))
            plt.plot(epochs, data['learning_rate'], label='Learning Rate')
            plt.title(f'Learning Rate vs. Epoch\n{model_name}')
            plt.xlabel('Epoch')
            plt.ylabel('Learning Rate')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.savefig(os.path.join(scalars_plot_dir, 'learning_rate.png'))
            plt.close()

        # Plot activations parameters dynamic
        act_params_stat_dir = os.path.join(model_log_dir, 'activations_parameters')
        if os.path.isdir(act_params_stat_dir):
            for file in os.listdir(act_params_stat_dir):
                file_path = os.path.join(act_params_stat_dir, file)
                data = np.load(file_path)
                if data is None or data.size == 0:
                    continue
                
                if data[0].size == 1:
                    alphas = data[:, 0]
                    betas = None
                else:
                    betas = data[:, 0]
                    alphas = data[:, 1]

                layer_name = file.removesuffix(".npy")
                
                
                # Plot params
                plt.figure(figsize=(8, 6))
                plt.plot(epochs, alphas, label='Alpha value')
                if betas is not None:
                    plt.plot(epochs, betas, label='Beta value')
                plt.title(f'Parameters vs. Epoch for {layer_name}\n{model_name}')
                plt.xlabel('Epoch')
                plt.ylabel('Value')
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.savefig(os.path.join(act_params_plot_dir, f'{layer_name}.png'))
                plt.close()
            
plot_histograms_and_scalars()

[LeNet5_hardtanh_quantized_2_bits] Unique activations of conv1.activation at Epoch 20 - 20632: [-1.         -0.9999734  -0.9999422  ...  0.99896157  0.9993932
  1.        ]
[LeNet5_hardtanh_quantized_2_bits] Unique activations of conv1.activation at Epoch 40 - 20359: [-1.         -0.9998843  -0.99974906 ...  0.9994586   0.99969643
  1.        ]
[LeNet5_hardtanh_quantized_2_bits] Unique activations of conv1.activation at Epoch 60 - 20413: [-1.         -0.99993503 -0.99975854 ...  0.99980456  0.9998784
  1.        ]
[LeNet5_hardtanh_quantized_2_bits] Unique activations of conv2.activation at Epoch 20 - 4: [-1.3333334 -0.6666667  0.         0.6666667]
[LeNet5_hardtanh_quantized_2_bits] Unique activations of conv2.activation at Epoch 40 - 4: [-1.3333334 -0.6666667  0.         0.6666667]
[LeNet5_hardtanh_quantized_2_bits] Unique activations of conv2.activation at Epoch 60 - 4: [-1.3333334 -0.6666667  0.         0.6666667]
[LeNet5_hardtanh_quantized_2_bits] Unique activations of fc1.activati